<a href="https://colab.research.google.com/github/Sourav1429/Restricted_BNN/blob/main/BNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pyro-ppl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 732.8/732.8 kB 5.9 MB/s eta 0:00:00


In [13]:
#------Code for Pulsar Dataset----------#
#------Standard BNN--------------#
# This code is motivated from: https://keras.io/examples/keras_recipes/bayesian_neural_networks/
import tensorflow as tf

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
import tensorflow_probability as tfp
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix

# Load the data
data = pd.read_excel('/content/Pulsar_reduced.xlsx')

# Assuming 'target' is the column containing the labels
X = data.drop('label', axis=1)
y = data['label']

# Handling missing values by filling with the mean
X = X.fillna(X.mean())

# Split the data into training, testing, and validation sets
# 80% for training, 10% for validation, and 10% for testing
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_validation, X_test, y_validation, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Scale the features using MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_validation_scaled = scaler.transform(X_validation)
X_test_scaled = scaler.transform(X_test)

# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential([
        tfp.layers.DistributionLambda(
            lambda t: tfp.distributions.MultivariateNormalDiag(
                loc=tf.zeros(n), scale_diag=tf.ones(n)
            )
        )
    ])
    return prior_model

# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential([
        tfp.layers.VariableLayer(
            tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
        ),
        tfp.layers.MultivariateNormalTriL(n),
    ])
    return posterior_model

def create_bnn_model(train_size):
    model = keras.Sequential()
    model.add(layers.InputLayer(input_shape=X_train_scaled.shape[1]))

    # Create hidden layers with weight uncertainty using the DenseVariational layer.
    for units in [2,2]:  #  number of units in hidden layers
        model.add(tfp.layers.DenseVariational(
            units=units,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation='relu',
        ))

    model.add(layers.Dense(units=1, activation='sigmoid'))
    return model

#  BNN model
bnn_model = create_bnn_model(train_size=len(X_train_scaled))

#  model with the Adam optimizer and binary cross-entropy loss
bnn_model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# training history
history = bnn_model.fit(X_train_scaled, y_train, validation_data=(X_validation_scaled, y_validation),
                         epochs=10, batch_size=32, verbose=0)


In [15]:
df = pd.DataFrame({'BNN':np.array(history.history['loss'])})
df.to_excel('BNN_pulsar.xlsx')

In [16]:
print("Accuracies:",history.history['accuracy'])

Accuracies: [0.48503291606903076, 0.7813809514045715, 0.8719816207885742, 0.9019157886505127, 0.9048094153404236, 0.9076032638549805, 0.9076032638549805, 0.9076032638549805, 0.9076032638549805, 0.9076032638549805]


In [7]:
# Plot epoch vs accuracy and loss
plt.figure(figsize=(12, 5))

# Plot Accuracy
#plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
#plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Iterations')
plt.ylabel('Accuracy')
#plt.title('Epoch vs Accuracy')
plt.legend()
plt.savefig('BNN_accuracy_plot.pdf', bbox_inches='tight')
plt.close()

# Plot Loss
#plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
#plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
#plt.title('Epoch vs Loss')
plt.legend()
plt.savefig('BNN_loss_plot.pdf', bbox_inches='tight')
plt.close()
plt.tight_layout()
plt.show()

# Make predictions on the test data
predictions_test = (bnn_model.predict(X_test_scaled) > 0.5).astype(int)

# Evaluate the model on the test set
accuracy_test = accuracy_score(y_test, predictions_test)
conf_matrix_test = confusion_matrix(y_test, predictions_test)

print("\nAccuracy on Test Set:", accuracy_test)
#print("\nConfusion Matrix on Test Set:\n", conf_matrix_test)

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 1.5]
 [6.9 3.1 4.